In [1]:
# from piec.drivers.utilities import PiecManager
from piec.drivers.keysight81150a import Keysight81150a
from piec.drivers.keysightdsox3024a import Dsox3024a
import pandas as pd
import time
# pm = PiecManager()
index = 10

In [2]:
# pm.parse_resources()

In [3]:
awg = Keysight81150a("GPIB0::8::INSTR")
scope = Dsox3024a("GPIB0::7::INSTR")

In [4]:
awg.idn()

'Agilent Technologies,81150A,MY53821602,3.0.0.0-4.6\n'

In [5]:
scope.idn()

'AGILENT TECHNOLOGIES,DSO-X 3024A,MY63080078,02.65.2021030741\n'

In [6]:
#Setup functions
#Set up the pulse params
awg.configure_wf(channel='1', func="PULS", voltage="2.0", offset="1.0", frequency="1.0e7", duty_cycle="50")
#Sets the trigger source to manual
awg.configure_trigger(trigger_source="MAN")
#turns the output on
awg.output_enable(channel='1')

In [22]:
#Configure the scope
vdiv = 0.02
channel=1

scope.initialize()
scope.configure_timebase(time_base_type='MAIN', reference='CENTer', time_scale=f'{5e-9}', position=f'{1.1e-7}') #this should be made general
scope.configure_channel(channel=f'{channel}', voltage_scale=f'{vdiv}', impedance='FIFT', voltage_offset=0)#set both to 50ohm
scope.configure_trigger_characteristics(trigger_source='EXT', low_voltage_level='0.75', holdoff_time='4E-7', high_voltage_level='0.95', sweep='NORM')
scope.configure_trigger_edge(trigger_source='EXT', input_coupling='DC', level=1.0)
scope.initiate()
time.sleep(1)
awg.send_software_trigger()

In [23]:
#Capture Waveform
scope.setup_wf()
_, trace_t, trace_v  = scope.query_wf()#change
data = pd.DataFrame({"time (s)":trace_t, "voltage (V)": trace_v}) # Retrieve the data from the oscilloscope
data.head()

,time (s),voltage (V)
0,8.500000e-08,-0.000804
1,8.507812e-08,-0.000804
2,8.515625e-08,-0.000804
3,8.523437e-08,0.000000
4,8.531250e-08,0.000000


In [24]:
name = f'{index}_10um_04.csv'
# name = f'{index}_shorted_probes.csv'

data.to_csv(r"C:\Users\adqua\Documents\Lab\data\mr0118\channel\\"+name, index=False)
index += 1